<a href="https://colab.research.google.com/github/waishun78/hungry-rag/blob/main/hungry_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installing the necessary packages
!pip install accelerate
!pip install bitsandbytes
!pip install SentencePiece
!pip install evaluate
!pip install bert_score
!pip install transformers

In [2]:
!nvidia-smi

Tue Oct 31 01:14:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Import necessary modules
import os
import torch
import torch.nn as nn
import bitsandbytes as bnb
import accelerate
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from transformers import LlamaTokenizer

In [4]:
load_in_8bit = True
model_name_l = "lmsys/vicuna-7b-v1.5"

model_l = AutoModelForCausalLM.from_pretrained(
    model_name_l,
    torch_dtype=torch.float16,
    load_in_8bit=load_in_8bit,
    device_map="auto"
)
tokenizer_l = LlamaTokenizer.from_pretrained(model_name_l)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [5]:
!pip install git+https://github.com/deepset-ai/haystack.git

  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-sa8j13l8
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-sa8j13l8
  Resolved https://github.com/deepset-ai/haystack.git to commit 76d5142bb8a03e38869ee86c4a1a9f098861db0b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [40]:
from haystack.nodes.base import BaseComponent

class Query_Rewriter(BaseComponent):
    outgoing_edges = 1

    def __init__(self, model, tokenizer):
      self.model = model
      self.tokenizer = tokenizer

    def run(self, query: str):

      prompt=f"""Generate some follow up questions I need to better answer your query
                  Example:
                  Question: Where to get Japanese food in Marina Bay Sands that is fine-dining and has a romantic atmosphere at night?
                  Answer:
                  1. What type of Japanese food are you looking for? e.g. sushi, ramen, teppanyaki, etc.
                  2. Do you have a preference for specific dishes or would you like to try a variety of options?
                  3. Are you looking for a specific price range for the food?
                  4. What are you looking for in a romantic atmsphere?
                  5. How late at night are you eating?
                  Question: {query}
                  Answer:"""
      input = self.tokenizer(prompt, return_tensors='pt')
      input_ids = input["input_ids"].to("cuda")
      temperature=0.7
      with torch.no_grad():
        generation_output = self.model.generate(
            input_ids=input_ids,
            temperature=temperature,
            top_p = 1.0,
            do_sample=True,
            return_dict_in_generate=True,
            max_new_tokens=200,
          )

      s = generation_output.sequences[0][len(input_ids[0]):]

      output = self.tokenizer.decode(s)
      return {"query":prompt, "output":output}, "output_1"

    def run_batch(self, queries):
        # Insert code here to manipulate the input and produce an output dictionary
        output = []
        for query in queries:
            output.append(self.run(query))
        return output, "output_1"

In [30]:
# Create Custom Query_Rewriter
query_rewriter = Query_Rewriter(model_l, tokenizer_l)

In [31]:
from haystack.pipelines import Pipeline

# Create a Haystack pipeline
p = Pipeline()
p.add_node(component=query_rewriter, name="QueryRewriter", inputs=["Query"])

In [42]:
query = "pizza near me"

In [39]:
result = p.run(query = query)

Generate some follow up questions I need to better answer your query 

                  Example:
                  Question: Where to get Japanese food in Marina Bay Sands that is fine-dining and has a romantic atmosphere at night?
                  

                  Answer:
                  1. What type of Japanese food are you looking for? e.g. sushi, ramen, teppanyaki, etc.
                  2. Do you have a preference for specific dishes or would you like to try a variety of options?
                  3. Are you looking for a specific price range for the food?
                  4. What are you looking for in a romantic atmsphere?
                  5. How late at night are you eating?
                  


                  Question: pizza near me
                  

                  Answer:
generated_output:SampleDecoderOnlyOutput(sequences=tensor([[    1,  3251,   403,   777,  1101,   701,  5155,   306,   817,   304,
          2253,  1234,   596,  2346, 29871,    13,    13,   

In [41]:
result

{'query': 'Generate some follow up questions I need to better answer your query \n\n                  Example:\n                  Question: Where to get Japanese food in Marina Bay Sands that is fine-dining and has a romantic atmosphere at night?\n                  \n\n                  Answer:\n                  1. What type of Japanese food are you looking for? e.g. sushi, ramen, teppanyaki, etc.\n                  2. Do you have a preference for specific dishes or would you like to try a variety of options?\n                  3. Are you looking for a specific price range for the food?\n                  4. What are you looking for in a romantic atmsphere?\n                  5. How late at night are you eating?\n                  \n\n\n                  Question: pizza near me\n                  \n\n                  Answer:',
 'output': '\n                  1. What type of pizza are you in the mood for? e.g. thin crust, deep dish, vegan, etc.\n                  2. Do you want to eat

In [56]:
updated_query = {
    "question":query,
    "clarifier": result["output"],
    "additional_requirements":
      """
      1. Chicago Pizza
      2. Indoors
      4. Nope
      """
}

In [57]:
class Annotate_Needs(BaseComponent):
    outgoing_edges = 1

    def __init__(self, model, tokenizer):
      self.model = model
      self.tokenizer = tokenizer

    def run(self, query: dict):

      prompt=f"""Generate the user's query requirements annotated with the requirement labels. Here are the categories:
                    Price, Opening, Location, Cuisine, Dietary Restrictions, Ratings, Reservations, Landmark, General (Atmosphere, idea-related needs)

                  Example:
                  Question: Where to get Japanese food in Marina Bay Sands that is fine-dining and has a romantic atmosphere at night?
                  Clarifying Questions:
                  1. What type of Japanese food are you looking for? e.g. sushi, ramen, teppanyaki, etc.
                  2. Do you have a preference for specific dishes or would you like to try a variety of options?
                  3. Are you looking for a specific price range for the food?
                  4. What are you looking for in a romantic atmsphere?
                  5. How late at night are you eating?
                  Additional User Criteria:
                  1. Sushi
                  2. Mostly just sushi
                  3. $$
                  4. Intimate with nice lighting, Inside Marina Bay Sands
                  5. 22:00 minimally
                  Answer:
                  Sushi[Cuisine], $$ [Price], Intimate with nice lighting [General-Atmosphere], Inside Marina Bay Sands [Location], 22:00 minimally [Opening]

                  ________________________________________________________________________________________________________
                  Question: {query["question"]}
                  Clarifying Questions: {query["clarifier"]}
                  Additional User Criteria:{query["additional_requirements"]}
                  Answer:"""
      input = self.tokenizer(prompt, return_tensors='pt')
      input_ids = input["input_ids"].to("cuda")
      temperature=0.7
      with torch.no_grad():
        generation_output = self.model.generate(
            input_ids=input_ids,
            temperature=temperature,
            top_p = 1.0,
            do_sample=True,
            return_dict_in_generate=True,
            max_new_tokens=200,
          )

      s = generation_output.sequences[0][len(input_ids[0]):]

      output = self.tokenizer.decode(s)
      return {"query":prompt, "output":output}, "output_1"

    def run_batch(self, queries):
        # Insert code here to manipulate the input and produce an output dictionary
        output = []
        for query in queries:
            output.append(self.run(query))
        return output, "output_1"

In [58]:
annotater = Annotate_Needs(model_l, tokenizer_l)

# Create a Haystack pipeline
p2 = Pipeline()
p2.add_node(component=annotater, name="Annotate_Needs", inputs=["Query"])

In [59]:
result2 = p2.run(query = updated_query)

In [60]:
print(result2)

{'query': "Generate the user's query requirements annotated with the requirement labels. Here are the categories:\n                    Price, Opening, Location, Cuisine, Dietary Restrictions, Ratings, Reservations, Landmark, General (Atmosphere, idea-related needs)  \n                  \n                  Example:\n                  Question: Where to get Japanese food in Marina Bay Sands that is fine-dining and has a romantic atmosphere at night?\n                  Clarifying Questions:                   \n                  1. What type of Japanese food are you looking for? e.g. sushi, ramen, teppanyaki, etc.\n                  2. Do you have a preference for specific dishes or would you like to try a variety of options?\n                  3. Are you looking for a specific price range for the food?\n                  4. What are you looking for in a romantic atmsphere?\n                  5. How late at night are you eating?\n                  Additional User Criteria:\n               